# Corpus retriever

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
#from textblob import TextBlob
#from textblob import Word
import requests
import lxml.etree
import json
import pymongo
import time

In [4]:
#Connections with collections
client = pymongo.MongoClient("mongodb://localhost:27017/")
mi_db = client["books"]
revsColec = mi_db["reviews"]

## Get reviews from Goodreads.com in XML

In [9]:
def getReviews():
    
    url='https://www.goodreads.com/review/recent_reviews.xml?&key=WhFX13ec5WJJSqQaXjuFWg'
    resp = requests.get(url)
    root = lxml.etree.fromstring(resp.content)
    
    # Review ID
    revIDs = [ item.text for item in root.xpath('//reviews/review/id') ]
    revRats = [ item.text for item in root.xpath('//reviews/review/rating') ]
    revVotess = [ item.text for item in root.xpath('//reviews/review/votes') ]
    revBodys = [ item.text for item in root.xpath('//reviews/review/body') ]
    revBodysOk =  [ re.sub( '<.*?>', '', str(revBody).strip() ) for revBody in revBodys ]
    
    userIDs = [ item.text for item in root.xpath('//reviews/review/user/id') ]
    userNames =  [ item.text for item in root.xpath('//reviews/review/user/name') ]
    userLocations = [ item.text for item in root.xpath('//reviews/review/user/location') ]
    userLocationsOk = [str(userLocation).split(', ')[-1] for userLocation in userLocations ]
    
    # Book info
    bookIDs = [ item.text for item in root.xpath('//reviews/review/book/id') ]
    bookISBNs = [ item.text for item in root.xpath('//reviews/review/book/isbn') ]
    bookISBN13s = [ item.text for item in root.xpath('//reviews/review/book/isbn13') ]
    bookTitles = [ item.text for item in root.xpath('//reviews/review/book/title') ]
    bookAuthorIDs = [ item.text for item in root.xpath('//reviews/review/book/authors/author/id') ]
    bookAuthorNames = [ item.text for item in root.xpath('//reviews/review/book/authors/author/name') ]
    bookFormats =  [ item.text for item in root.xpath('//reviews/review/book/format') ]
    bookNpagess = [ item.text for item in root.xpath('//reviews/review/book/num_pages') ]
    bookAvgRatings = [ item.text for item in root.xpath('//reviews/review/book/average_rating') ]
    bookRatCounts = [ item.text for item in root.xpath('//reviews/review/book/ratings_count') ]
    
    newInfo = {
                '_id': revIDs, 'revRat': revRats, 'revVotes': revVotess, 'revBody': revBodysOk, 'revRats': revRats,

                'userID': userIDs, 'userName': userNames, 'userLocation': userLocationsOk,
                'bookID': bookIDs,  'bookISBN': bookISBNs, 'bookISBN13': bookISBN13s, 'bookTile': bookTitles,
                'bookAuthorID': bookAuthorIDs, 'bookAuthorName': bookAuthorNames, 'bookFormat': bookFormats, 
                'bookNpages': bookNpagess, 'bookAvgRating': bookAvgRatings, 'bookRatCount': bookRatCounts
                }

    return newInfo

In [10]:
reviewsDF = pd.DataFrame(getReviews())
reviewsDF

,_id,revRat,revVotes,revBody,revRats,userID,userName,userLocation,bookID,bookISBN,bookISBN13,bookTile,bookAuthorID,bookAuthorName,bookFormat,bookNpages,bookAvgRating,bookRatCount
0,3325240005,4,0,I received this book as an advanced copy &amp;...,4,49081817,Lisa,None,53216402,None,None,"Cowboy's Retreat (Fated to Cowboy, #3)",13775484,Ella Cooper,ebook,70,4,19
1,3325239472,5,0,This clean western was a splendid additionSo h...,5,60147675,Sandy,FL,53214345,None,None,"A Secretive Bride (The Brides of Briar Cliff, #5)",19998167,Kellie Abbott,Kindle Edition,None,4,7
2,3325238810,4,0,A fantastic story. Every tense step is wonderf...,4,40189319,Ian,Canada,45046527,None,None,American Dirt,338667,Jeanine Cummins,Hardcover,400,4,44563
3,3325238653,5,0,"this is a great read, an alien romance filled ...",5,23795159,Melody,IA,53332641,None,None,Jave (Marauder Mates Book 5),8145962,Athena Storm,Kindle Edition,203,4,22
4,3325238247,4,0,"Family, love, revenge and redemption. For Deni...",4,6562569,Angela,None,53274745,None,None,"Hart of Vengeance (The Hart, #2)",6871111,S.B. Alexander,None,None,4,28
5,3325238029,4,0,From the perspective of Tsunami. You get to vi...,4,27305391,Alexis Mobley,The United States,15809617,0545349192,9780545349192,"The Lost Heir (Wings of Fire, #2)",374504,Tui T. Sutherland,Hardcover,296,4,15519
6,3325237850,5,0,"This coloring book is directed at boy, but cou...",5,11945045,Betty,CO,53349776,B088B3MPLT,9798643658351,"I Am Kind, Strong And Smart",17047430,Jacob Mason,Paperback,32,0,0
7,3325237685,5,0,Excellent read after The Glass Castle - Two of...,5,73795629,Leslie,GA,6366437,1416586288,9781416586289,Half Broke Horses,3275,Jeannette Walls,Hardcover,272,4,125081
8,3325237662,5,0,"great book, solid intro. Recommend reading Sta...",5,33848298,Matias Andrade,Peru,17571613,None,None,Network Science,2932657,Albert-László Barabási,ebook,None,4,53
9,3325237414,5,0,This was one of the best duets I've ever read....,5,29676909,Tianna,WA,53226558,None,None,Fae Unchained (Mage Shifter War #2),15042800,Elle Middaugh,Kindle Edition,None,4,99


## Get Descriptions from Goodreads.com

In [4]:
def getDescription(bookID):
    
    if bookID is None:
        return None
    else:
        url='https://www.goodreads.com/book/show/' + str(bookID) + '?key=WhFX13ec5WJJSqQaXjuFWg'
        resp = requests.get(url)
        root = lxml.etree.fromstring(resp.content)
    
        bookDescriptions = [ item.text for item in root.xpath('//book/description') ]
        bookDescriptionsOk =  [ re.sub( '<.*?>', '', str(bookDescription).strip() ) for bookDescription in bookDescriptions ]
        
        newInfo =  bookDescriptionsOk[0]
    
        return newInfo

## Get some prices from Booksrun.com

In [6]:
def getPrice(isbn13):
    
    if isbn13 is None:
        return None
    else:
        url= 'https://booksrun.com/api/v3/price/buy/' + str(isbn13) + '?key=hioqkouwix69eby707e8'
        resp = requests.get(url)
        resul = json.loads(resp.content)
        try:
            return resul['result']['offers']['booksrun']['new']['price']
        except:
            return None

## Get all the info using the previous functions

In [8]:
def getInfo():
    
    reviewsDF = pd.DataFrame(getReviews())
    exceptions = revsColec.distinct("_id")
    reviewsDF = reviewsDF[~reviewsDF['_id'].isin(exceptions)] 
    
    reviewsDF['bookDescription'] = [getDescription(str(item)) for item in reviewsDF['bookID'] ]
    reviewsDF['Price'] = [getPrice(str(item)) for item in reviewsDF['bookISBN13'] ]
    
    return reviewsDF

## Write the info in mongoDB collection

In [9]:
def DBinsertion(DF):
    
    dicc = DF.T.to_dict("dict")
    for v in list(dicc.values()):
        try:
            revsColec.insert_one(v)
        except:
            continue

In [10]:
def DBwriteInfo(k, timeLapse = 60):
    
    n = k
    while True:
        try:
            print('Batches remaining: ', n)
            DF = getInfo()
            DBinsertion(DF)
            time.sleep(timeLapse) 
            n = n - 1
            if n == 0:
                print('Done')
                break
        except:
            print('Batch failed')
            time.sleep(timeLapse)
            continue

In [ ]:
DBwriteInfo(100)

Batches remaining:  100
Batches remaining:  99
Batches remaining:  98
Batches remaining:  97
Batches remaining:  96
Batches remaining:  95
Batches remaining:  94
Batches remaining:  93
Batches remaining:  92
Batches remaining:  91
Batches remaining:  90
Batches remaining:  89
Batches remaining:  88
Batches remaining:  87
Batches remaining:  86
Batches remaining:  85
Batches remaining:  84
Batches remaining:  83
Batches remaining:  82
Batches remaining:  81
Batches remaining:  80
Batches remaining:  79
Batches remaining:  78
Batches remaining:  77
Batches remaining:  76
Batches remaining:  75
Batches remaining:  74
Batches remaining:  73
Batches remaining:  72
Batches remaining:  71
